# SQL Review and Practice


Many of the examples have been adapted from Data100 and J. Canny's course.

## Setup 

This notebook makes use of a python module: `utils.py` as well as 4 data files: 

* `pls_fy2009_pupld09a.csv` 
* `pls_fy2014_pupld09a.csv` 
* `us_counties_2000.csv`
* `us_counties_2010.csv` 


In [2]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

#from pathlib import Path
# !pip install sqlalchemy
#from sqlalchemy import create_engine
from utils import fetch_and_cache

In [3]:
%%html
<style>
table {margin-left: 0 !important;}
</style>

For this lab, we will be using SQLite to connect to databases. This is a simple, light-way module, there are of course other modules to interact with databases. 

SQLite has a [nice tutorial](https://www.sqlitetutorial.net/) if you want to learn more about database queries and explore additional options. 

In [4]:
import sqlite3

A couple of things to know about SQLite.  Other database systems such as `MySQL` and `PostgreSQL` use *static typing*, where a column is declared for a specific data type and can only store data of that type. 

SQLite uses *dynamic type system*, where a value stored in a column determines its data type, not the column's data type.  

# Connecting to a Database

## Example 1 - School Enrollment

*Example from Practical SQL, by Anthony DeBarros, 2018 - Available on O'reilly's learning platform*  
Let's look at an example that walks through a creating a table, inserting rows, querying the table, etc. 

For this example, we are going to create a database for managing a school's enrollment.  Within the database there are several tables. 

The first table, `student_enrollment`, shows student signed up for each class section: 

    student_id    class_id      class_section    semester
    ----------    ----------    -------------    ---------
    CHRISPA004    COMPSCI101    3                Fall 2017
    DAVISHE010    COMPSCI101    3                Fall 2017
    ABRILDA002    ENG101        40               Fall 2017
    DAVISHE010    ENG101        40               Fall 2017
    RILEYPH002    ENG101        40               Fall 2017

A separate table, `students`, stores information about the students. The `student_id` column is used to identify each student.  The value is the unique `key` to connect the tables.

    student_id    first_name    last_name    dob       
    ----------    ----------    ---------    ----------
    ABRILDA002    Abril         Davis        1999-01-10
    CHRISPA004    Chris         Park         1996-04-10
    DAVISHE010    Davis         Hernandez    1987-09-14
    RILEYPH002    Riley         Phelps       1996-06-15

A final table, `classes`, stores information about each class, where the `class_id` is the primary key. 

    class_id      first_name    last_name
    ----------    ----------    ---------
    COMPSCI101    Davis         Hernandez
    COMPSCI101    Chris         Park
    ENG101        Abril         Davis
    ENG101        Davis         Hernandez
    ENG101        Riley         Phelps

Let's connect to a new, empty database `analysis.db`

In [5]:
conn = sqlite3.connect("analysis.db")

### Printing out SQL calls and results 

We define a function `print_sql(s)` that given a database connection `c` and a sql query `s` returns the result of that executing.

In [6]:
def print_sql(c, s):
    print('>', s)
    for result in c.execute(s):
        print(result)
    print()

We can confirm that the database is empty. 

In [7]:
print_sql(conn, 'SELECT sql FROM sqlite_master;')

> SELECT sql FROM sqlite_master;



We could also use `pandas` `read_sql` function to read in the results of the query to a dataframe to display.  

*Be careful with running this code as it will create the dataframe - for a query that returns many records this could be memory-intensive*

In [8]:
def pretty_print_sql(c, s):
    print('>', s)
    df = pd.read_sql(s, c)
    display(df)

In [9]:
pretty_print_sql(conn, 'SELECT sql FROM sqlite_master;')

> SELECT sql FROM sqlite_master;


,sql


# Create a Relation (Table) 



The simple form of creating a table is: 

```mysql
CREATE TABLE <name> (
    <alist of elements>
);
```

## Example 2 - Banking

Banking example from J. Freire consisting of the following relations. 

The first table, `Account`, has banking account information: 

    number    custID    owner        balance      type
    ------    ------    ---------    ---------    ---------
    101       1         J. Smith     1000.00      checking
    102       2         W. Wei       2000.00      checking
    103       1         J. Smith     5000.00      saving
    104       3         M. Jones     1000.00      checking
    105       4         H. Martin    10000.00     checking

The second table, `Deposit`, has transaction information: 

    account    transID    dDate       amount
    -------    -------    --------    ---------
    102        1          10/22/21    500.00
    102        2          10/29/21    200.00
    104        3          10/29/21    1000.00
    105        4          11/2/21     10000.00

The third table `CheckInfo`, has information on checks written: 

    account    checkNum    cDate       amount
    -------    --------    --------    ---------
    101        924         10/23/21    125.00
    101        925         10/24/21    23.98

#### Elements of Table Declarations 

The list of elements in a table declaration are at minimum an attribute (column) and its type.

The most common types are: 
* INT or INTEGER 
* REAL or FLOAT 
* CHAR(*n*)   = fixed-length string of *n* characters
* VARCHAR(*n*) = variable-length string of up to *n* characters


#### SQL Values 

* Integer and  reals are represented as you would expect. 
* Strings are too, except they require single quotes. 
    & Two single quotes  = real quote, e.g., `‘Ed’’s Bookstore’`
* Any value can be NULL!

Dates and Times 
* DATE and TIME are types in SQL. 
* The form of a date value is:  
    DATE ‘yyyy-mm-dd’  
    Example: `DATE '2022-10-12'`
* The form of a time value is:  
    TIME ‘hh:mm:ss’
  with an optional decimal point and fractions of a second following.   
    Example:  `TIME ’15:30:02.5'`

#### Declaring Keys 

* An attribute or list of attributes may be declared PRIMARY KEY or UNIQUE
* Either says that no two tuples of the relation may agree in all the attribute(s) on the list
* There are a few distinctions to be mentioned later

##### Declaring Single-Attribute Keys

* Place PRIMARY KEY or UNIQUE after the type in the declaration of the attribute.
* Example: 
```mysql 
CREATE TABLE Account(
    number CHAR(20) PRIMARY KEY, 
    custID INTEGER,
    owner VARCHAR(30),
    balance REAL,
    type CHAR(10)
);
```
or 
```mysql 
CREATE TABLE Account(
    number CHAR(20) UNIQUE, 
    custID INTEGER,
    owner VARCHAR(30),
    balance REAL,
    type CHAR(10)
);
```
or 
```mysql 
CREATE TABLE Account(
    number CHAR(20), 
    custID INTEGER,
    owner VARCHAR(30),
    balance REAL,
    type CHAR(10), 
    PRIMARY KEY (number)
);
```

##### Declaring Multi-Attribute Keys

* A key declaration can also be another element in the list of elements of a CREATE TABLE statement. 
* This form is essential if the key consists of more than one attribute.
    * May be used even for one-attribute keys
* In the `CheckInfo` table the `account` and `checkNum` are the key. 

```mysql 
CREATE TABLE CheckInfo (
    account CHAR(20), 
    checkNum INT,
    cDate DATE,
    amount REAL, 
    PRIMARY KEY (account, checkNum)
);
```

#### PRIMARY KEY vs. UNIQUE 

1. There can be only one PRIMARY KEY for a relation, but several UNIQUE attributes.
2. No attribute of a PRIMARY KEY can ever be NULL in any tuple. But attributes declared UNIQUE may have NULL’s, and there may be several tuples with NULL.

### Example 2 - Create Tables

In [10]:
# Example 2 - Banking 
# Create a connection to the database "banking.db"
conn2 = sqlite3.connect("banking.db")

# Add the three tables for Account, Deposit and CheckInfo
conn2.executescript("""
CREATE TABLE Account (
    number INTEGER PRIMARY KEY,
    custID INTEGER,
    owner TEXT,
    balance REAL,
    type TEXT
);
""");

conn2.executescript("""
CREATE TABLE Deposit (
    accountID INTEGER,
    transID INTEGER PRIMARY KEY, 
    dDate TEXT, 
    amount REAL, 
    FOREIGN KEY (accountID)
       REFERENCES Account (number)
);
""")

conn2.executescript("""
CREATE TABLE CheckInfo (
    accountID INTEGER NOT NULL,
    checkNum INTEGER NOT NULL, 
    cDate TEXT,
    amount REAL,
    PRIMARY KEY (accountID, checkNum), 
    FOREIGN KEY (accountID)
       REFERENCES Account (number)
);
""")

### Different Database Systems

Below, I list the differences in syntax between database systems.  Note, I will not do this for all queries, but want to highlight the commonalities and small differences. 

#### SQLite Syntax 

Let's create a table in the `analysis` database from **Example 1** above.   

```mysql
CREATE TABLE teachers (
    id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
    first_name TEXT,
    last_name TEXT,
    school TEXT,
    hire_date TEXT,
    salary INTEGER
);
```

The table `teachers`  stores 6 pieces of information: 

* `id` of type `integer`, that auto-increments each time you add a row to the table. 
* `first_name` is type `TEXT` a text column. 
* `last_name`, a text column. 
* `school`, a text column. 
* `hire_date`, is set to type `TEXT`, there is no date and time type in SQLite. 
* `salary` is type INTEGER 


#### PostgreSQL Syntax

Let's create a table in the `analysis` database from Example 1 above.   

```mysql
CREATE TABLE teachers (
    id bigserial,
    first_name varchar(50),
    last_name varchar(50),
    school varchar(50),
    hire_date date,
    salary numeric
);
```

The table `teachers`  stores 6 pieces of information: 

* `id` of type `bigserial`, that auto-increments each time you add a row to the table. 
* `first_name` is type `varchar` a text column with a maximum length specified. 
* `last_name`, a text column limited to 50 characters. 
* `school`, a text column for 50 characters 
* `hire_date`, is set to type `date` 
* `salary` is type numeric 

In [11]:
# Example 1 - School Enrollment
# Create tables in the "analysis.db" database
conn.executescript("""
CREATE TABLE teachers (
    id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
    first_name TEXT,
    last_name TEXT,
    school TEXT,
    hire_date TEXT,
    salary INTEGER
);
""");

### Insert rows into a Table 

Many times we will want to automatically insert a large number or rows directly from a text file or another database.  Here, we will add a few rows using an `INSERT INTO ... VALUES` statement.  

```mysql 
INSERT INTO teachers (first_name, last_name, school, hire_date, salary)
VALUES ('Janet', 'Smith', 'F.D. Roosevelt HS', '2011-10-30', 36200),
       ('Lee', 'Reynolds', 'F.D. Roosevelt HS', '1993-05-22', 65000),
       ('Samuel', 'Cole', 'Myers Middle School', '2005-08-01', 43500),
       ('Samantha', 'Bush', 'Myers Middle School', '2011-10-30', 36200),
       ('Betty', 'Diaz', 'Myers Middle School', '2005-08-30', 43500),
       ('Kathleen', 'Roush', 'F.D. Roosevelt HS', '2010-10-22', 38500);
``` 

In [12]:
# Example 1 - School Enrollment
# Add rows to the teachers table 
conn.executescript("""
INSERT INTO teachers (first_name, last_name, school, hire_date, salary)
VALUES ('Janet', 'Smith', 'F.D. Roosevelt HS', '2011-10-30', 36200),
       ('Lee', 'Reynolds', 'F.D. Roosevelt HS', '1993-05-22', 65000),
       ('Samuel', 'Cole', 'Myers Middle School', '2005-08-01', 43500),
       ('Samantha', 'Bush', 'Myers Middle School', '2011-10-30', 36200),
       ('Betty', 'Diaz', 'Myers Middle School', '2005-08-30', 43500),
       ('Kathleen', 'Roush', 'F.D. Roosevelt HS', '2010-10-22', 38500);
""");

We can also add rows one at a time. 

In [13]:
# Example 1 - School Enrollment
# Add a single row to the teachers table
conn.executescript("""
INSERT INTO teachers (first_name, last_name, school, hire_date, salary) 
VALUES ('Hank', 'Smith', 'Jefferson HS', '2018-01-01', 32000);
""");

#### Example 2 - Add Data 

Populate tables, `Account`, `Deposit`, `CheckInfo` and `ATMwithdraw`.  The `ATMwithdraw` has the following schema `ATMwithdraw(transID, custID, account, amount, wDate)` where `transID` is the primary key.   

In [14]:
# Example 2 
# Add data to populate tables

conn2.executescript("""
INSERT INTO Account (number, custID, owner, balance, type)
VALUES (101, 1, 'J. Smith', 1000.00, 'checking'), 
       (102, 2, 'W.Wei', 2000.00, 'checking'),
       (103, 1, 'J. Smith', 5000.00, 'saving'), 
       (104, 3, 'M. Jones', 1000.00, 'checking'),
       (105, 4, 'H. Martin', 10000.00, 'checking');
""")

conn2.executescript("""
INSERT INTO Deposit (accountID, transID, dDate, amount)
VALUES (102, 1, '10/22/21', 500.00), 
       (102, 2, '10/29/21', 200.00), 
       (104, 3, '10/29/21', 1000.00), 
       (105, 4, '11/2/21', 10000.00);
""")

conn2.executescript("""
INSERT INTO CheckInfo (accountID, checkNum, cDate, amount)
VALUES (101, 924, '10/23/21', 125.00),
       (101, 925, '10/24/21', 23.98);
""")

conn2.executescript("""
CREATE TABLE ATMwithdraw (
    transID INTEGER NOT NULL,
    custID INTEGER, 
    accountID INTEGER,
    amount REAL,
    wDate TEXT,
    PRIMARY KEY (transID), 
    FOREIGN KEY (accountID)
       REFERENCES Account (number)
    FOREIGN KEY (custID)
       REFERENCES Account (custID)
);
""")

conn2.executescript("""
INSERT INTO ATMwithdraw (transID, custID, accountID, amount, wDate)
VALUES (1, 2, 102, 25.00, '11/01/21 09:45:00'), 
       (2, 2, 102, 150.00, '11/10/2021 13:15:00'), 
       (3, 1, 101, 40.00, '11/01/2021 10:05:00'), 
       (4, 1, 101, 40.00, '11/01/2021 10:07:00'), 
       (5, 1, 101, 200.00, '11/8/2021 14:14:00');
""")

# SQL Queries 

## Select-From-Where Statements

Select-From-Where Statements are a common form of queries. 

``` 
SELECT  desired attributes 
FROM    one or more tables 
WHERE   condition about tuples of the tables
```

#### Inspect every row and column 

Let's look to ensure the table is as expected. 

First a `SELECT` statement to fetch every row and column in a table called `my_table`. 

```mysql
SELECT * FROM my_table;
```



##### Example 1 - Select Query

We can use our functions to run a specific on the database connection and display the results.

In [15]:
print_sql(conn, 'SELECT * FROM teachers;')

> SELECT * FROM teachers;
(1, 'Janet', 'Smith', 'F.D. Roosevelt HS', '2011-10-30', 36200)
(2, 'Lee', 'Reynolds', 'F.D. Roosevelt HS', '1993-05-22', 65000)
(3, 'Samuel', 'Cole', 'Myers Middle School', '2005-08-01', 43500)
(4, 'Samantha', 'Bush', 'Myers Middle School', '2011-10-30', 36200)
(5, 'Betty', 'Diaz', 'Myers Middle School', '2005-08-30', 43500)
(6, 'Kathleen', 'Roush', 'F.D. Roosevelt HS', '2010-10-22', 38500)
(7, 'Hank', 'Smith', 'Jefferson HS', '2018-01-01', 32000)



In [16]:
pretty_print_sql(conn, 'SELECT * FROM teachers;')

> SELECT * FROM teachers;


,id,first_name,last_name,school,hire_date,salary
0,1,Janet,Smith,F.D. Roosevelt HS,2011-10-30,36200
1,2,Lee,Reynolds,F.D. Roosevelt HS,1993-05-22,65000
2,3,Samuel,Cole,Myers Middle School,2005-08-01,43500
3,4,Samantha,Bush,Myers Middle School,2011-10-30,36200
4,5,Betty,Diaz,Myers Middle School,2005-08-30,43500
5,6,Kathleen,Roush,F.D. Roosevelt HS,2010-10-22,38500
6,7,Hank,Smith,Jefferson HS,2018-01-01,32000


##### Example 2 - Select Query

Here we can use the same query structure for Example 2 to access the `ATMwithdraw` table. 

In [17]:
pretty_print_sql(conn2, 'SELECT * FROM ATMwithdraw')

> SELECT * FROM ATMwithdraw


,transID,custID,accountID,amount,wDate
0,1,2,102,25.0,11/01/21 09:45:00
1,2,2,102,150.0,11/10/2021 13:15:00
2,3,1,101,40.0,11/01/2021 10:05:00
3,4,1,101,40.0,11/01/2021 10:07:00
4,5,1,101,200.0,11/8/2021 14:14:00


#### Querying a Subset of Columns

With large tables, we may want to only look at a subset of information.  We can replace the `*` with a list of columns separated by columns.  For example, 

```mysql 
SELECT some_column, another_column, more_columns 
FROM table_name;
```

This will retrieve all rows from just those three columns. 

In [18]:
print_sql(conn, 'SELECT last_name, first_name, salary FROM teachers;')

> SELECT last_name, first_name, salary FROM teachers;
('Smith', 'Janet', 36200)
('Reynolds', 'Lee', 65000)
('Cole', 'Samuel', 43500)
('Bush', 'Samantha', 36200)
('Diaz', 'Betty', 43500)
('Roush', 'Kathleen', 38500)
('Smith', 'Hank', 32000)



##### Using DISTINCT to Find Unique Values

In a table, it is not unusual for a column to contain rows with duplicate values.  We can use the keyword `DISTINCT` to eliminate duplicates, that is, show only unique values. 

In [19]:
print_sql(conn, 'SELECT school FROM teachers;')

> SELECT school FROM teachers;
('F.D. Roosevelt HS',)
('F.D. Roosevelt HS',)
('Myers Middle School',)
('Myers Middle School',)
('Myers Middle School',)
('F.D. Roosevelt HS',)
('Jefferson HS',)



In [20]:
print_sql(conn, 'SELECT DISTINCT school FROM teachers;')

> SELECT DISTINCT school FROM teachers;
('F.D. Roosevelt HS',)
('Myers Middle School',)
('Jefferson HS',)



The distinct keyword also works on more than one column at a time. 

In [21]:
print_sql(conn, 'SELECT DISTINCT school, salary FROM teachers;')

> SELECT DISTINCT school, salary FROM teachers;
('F.D. Roosevelt HS', 36200)
('F.D. Roosevelt HS', 65000)
('Myers Middle School', 43500)
('Myers Middle School', 36200)
('F.D. Roosevelt HS', 38500)
('Jefferson HS', 32000)



#### Sorting Data with ORDER BY 

Results of a query can be ordered using the keywords `ORDER BY`.  Note, this does not change the original table, only the query result. 

```mysql 
SELECT first_name, last_name, salary 
FROM teachers 
ORDER BY salary DESC;
```

In [22]:
print_sql(conn, 
          'SELECT first_name, last_name, salary FROM teachers ORDER BY salary DESC;')

> SELECT first_name, last_name, salary FROM teachers ORDER BY salary DESC;
('Lee', 'Reynolds', 65000)
('Samuel', 'Cole', 43500)
('Betty', 'Diaz', 43500)
('Kathleen', 'Roush', 38500)
('Janet', 'Smith', 36200)
('Samantha', 'Bush', 36200)
('Hank', 'Smith', 32000)



We can also sort by multiple columns. For example, here we are sorting by `school` in ascending order and `hire_date` in descending order. 

Note, we are also making use of a string with the triple quotations, `"""` that allows us to create a multiline string.  This is nice to use when formatting SQL queries. 

In [23]:
print_sql(conn, """
SELECT last_name, school, hire_date 
FROM teachers 
ORDER BY school ASC, hire_date DESC;
""")

> 
SELECT last_name, school, hire_date 
FROM teachers 
ORDER BY school ASC, hire_date DESC;

('Smith', 'F.D. Roosevelt HS', '2011-10-30')
('Roush', 'F.D. Roosevelt HS', '2010-10-22')
('Reynolds', 'F.D. Roosevelt HS', '1993-05-22')
('Smith', 'Jefferson HS', '2018-01-01')
('Bush', 'Myers Middle School', '2011-10-30')
('Diaz', 'Myers Middle School', '2005-08-30')
('Cole', 'Myers Middle School', '2005-08-01')



### Filtering Rows with WHERE 

We may want to limit the rows returned to those that meet a certain criteria. 
Using teachers as an example, we might want to find all teachers hired before a particular year or all teachers making more than $75,000 at elementary schools.

```mysql 
SELECT last_name, school, hire_date
FROM teachers
WHERE school = 'Myers Middle School';
```

In [24]:
print_sql(conn, """SELECT last_name, school, hire_date 
FROM teachers 
WHERE school = 'Myers Middle School';""")

> SELECT last_name, school, hire_date 
FROM teachers 
WHERE school = 'Myers Middle School';
('Cole', 'Myers Middle School', '2005-08-01')
('Bush', 'Myers Middle School', '2011-10-30')
('Diaz', 'Myers Middle School', '2005-08-30')



There are a number of different comparison operators that can be used (we have `=` above).  The table below lists several. 

| Operator  | Function  | Example      | 
|-----------|-----------|--------------|
| `=`       | Equal to  | `WHERE school = 'Baker Middle` | 
| `<>` or `!=` | Not equal to | `WHERE school <> 'Baker Middle` | 
| `>`       | Greater than | `WHERE salary > 20000` |
| `<`       | Less than | `WHERE salary < 60000` | 
| `>=`      | Greater than or equal to  | `WHERE salary >= 20000` | 
| `<=`      | Less than or equal to     | `WHERE salary <= 60000` | 
| `BETWEEN` | Within a range  | `WHERE salary BETWEEN 20000 AND 40000` | 
| `IN`      | Match one of a set of values | `WHERE last_name IN ('Bush', 'Roush')` | 
| `LIKE`    | Match a pattern (case sensitive) | `WHERE first_name LIKE 'Sam%'` | 
| `NOT`     | Negates a condition | `WHERE first_name NOT ILIKE 'sam%'` | 


In [25]:
print_sql(conn, """
SELECT first_name, last_name, school
FROM teachers
WHERE first_name = 'Janet';
""")

> 
SELECT first_name, last_name, school
FROM teachers
WHERE first_name = 'Janet';

('Janet', 'Smith', 'F.D. Roosevelt HS')



In [26]:
print_sql(conn, """
SELECT school
FROM teachers
WHERE school != 'F.D. Roosevelt HS';
""")

> 
SELECT school
FROM teachers
WHERE school != 'F.D. Roosevelt HS';

('Myers Middle School',)
('Myers Middle School',)
('Myers Middle School',)
('Jefferson HS',)



In [27]:
print_sql(conn, """
SELECT first_name, last_name, hire_date
FROM teachers
WHERE hire_date < '2000-01-01';
""")

> 
SELECT first_name, last_name, hire_date
FROM teachers
WHERE hire_date < '2000-01-01';

('Lee', 'Reynolds', '1993-05-22')



In [28]:
print_sql(conn, """
SELECT first_name, last_name, school, salary
FROM teachers
WHERE salary BETWEEN 40000 AND 65000;
""")

> 
SELECT first_name, last_name, school, salary
FROM teachers
WHERE salary BETWEEN 40000 AND 65000;

('Lee', 'Reynolds', 'F.D. Roosevelt HS', 65000)
('Samuel', 'Cole', 'Myers Middle School', 43500)
('Betty', 'Diaz', 'Myers Middle School', 43500)



Select tuples that match a condition from Example 2. 

In [29]:
print_sql(conn2, """
SELECT * 
FROM ATMWithdraw 
WHERE amount < 50;
""")

> 
SELECT * 
FROM ATMWithdraw 
WHERE amount < 50;

(1, 2, 102, 25.0, '11/01/21 09:45:00')
(3, 1, 101, 40.0, '11/01/2021 10:05:00')
(4, 1, 101, 40.0, '11/01/2021 10:07:00')



##### Using LIKE  with WHERE 

When using `LIKE`, you can search for patterns in strings using two special characters: 

* **Percent sign %** A wildcard matching one or more characters 
* **Underscore _** A wildcard matching exactly one character 

For example the following patterns with find the word `baker` 

```mysql 
LIKE 'b%'
LIKE '%ak%'
LIKE '_aker'
LIKE 'ba_er'
```

In [30]:
print_sql(conn, """
SELECT first_name
FROM teachers
WHERE first_name LIKE 'sam%';
""")

> 
SELECT first_name
FROM teachers
WHERE first_name LIKE 'sam%';

('Samuel',)
('Samantha',)



##### Combining operators with AND and OR

Comparison operators can be combined with `AND` and `OR` with parentheses if needed. 

In [31]:
print_sql(conn, """
SELECT *
FROM teachers
WHERE school = 'F.D. Roosevelt HS'
      AND (salary < 38000 OR salary > 40000);
""")

> 
SELECT *
FROM teachers
WHERE school = 'F.D. Roosevelt HS'
      AND (salary < 38000 OR salary > 40000);

(1, 'Janet', 'Smith', 'F.D. Roosevelt HS', '2011-10-30', 36200)
(2, 'Lee', 'Reynolds', 'F.D. Roosevelt HS', '1993-05-22', 65000)



### Connections to Relational Algebra 

There can be a basic syntax comparison between relational algebra operators and SQL. 

####  Projection 

The projection operator $\pi_{a,b}$ is related to `SELECT a, b` 

In [32]:
print_sql(conn2, """
SELECT custID, amount 
FROM ATMWithdraw 
""")

> 
SELECT custID, amount 
FROM ATMWithdraw 

(2, 25.0)
(2, 150.0)
(1, 40.0)
(1, 40.0)
(1, 200.0)



#### Selection 

The selection operator $\sigma_{(d > e) \wedge (f = g)}$ is related to `WHERE d > e AND f = g`

In [33]:
print_sql(conn2, """
SELECT * 
FROM ATMWithdraw 
WHERE (amount > 50) AND (accountID==102)
""")

> 
SELECT * 
FROM ATMWithdraw 
WHERE (amount > 50) AND (accountID==102)

(2, 2, 102, 150.0, '11/10/2021 13:15:00')



#### Renaming 

Renaming attributes or $\rho$ relates to `AS {or blank space}`. 

Here we can rename the `custID` attribute as `owner`


In [34]:
pretty_print_sql(conn2, """
SELECT custID AS owner, amount 
FROM ATMWithdraw 
""")

> 
SELECT custID AS owner, amount 
FROM ATMWithdraw 



,owner,amount
0,2,25.0
1,2,150.0
2,1,40.0
3,1,40.0
4,1,200.0


### NULL Values 

Tuples in SQL relations can have NULL as a value for one or more components.  What a NULL value means depends on the context, but two common cases are: 

* _Missing value_:  we don't know a particular attribute's value for a row. 
* _Inapplicable_: e.g., the value of the attribute `spouse` for an unmarried person.  

The logic on conditions in SQL is really **3-valued logic**: TRUE, FALSE, UNKNOWN. 

Comparing any value (including NULL itself) with NULL yields UNKNOWN.  

A tuple is in a query answer if and only if the WHERE clause is TRUE (not FALSE or UNKNOWN).  

#### Three-Valued Logic 

To understand how AND, OR, and NOT work in 3-valued logic, think of: 

* TRUE = 1, FALSE = 0, and UNKNOWN = 1/2 
* AND = MIN, OR = MAX, NOT(x) = 1 - x. 

For example, 

$$ \text{TRUE AND (FALSE OR NOT(UNKNOWN))} \\ 
 = MIN(1, MAX(0, (1-1/2)) \\ 
 = MIN(1, MAX(0, 1/2))  \\
 = MIN(1, 1/2) = 1/2 $$ 
 

### Importing Data 

Many times we might have data from a delimited file.  We can import or export this data. 

In SQLite, this is done at the command line with the `.import` command.  In PostgreSQL, this is done with the `COPY` command. 

Here we can take advantage of `pandas`, to write a DataFrame to an sql database with the `to_sql` function. 

In [35]:
df = pd.read_csv("us_counties_2010.csv")
df.to_sql('us_counties_2010', conn, if_exists='append', index=False)
df.head()

,NAME,STUSAB,SUMLEV,REGION,DIVISION,STATE,COUNTY,AREALAND,AREAWATR,POP100,...,P0040010,P0040011,P0040012,P0040028,P0040049,P0040065,P0040072,H0010001,H0010002,H0010003
0,Autauga County,AL,50,3,6,1,1,1539582278,25775735,54571,...,22,384,363,19,2,0,0,22135,20221,1914
1,Baldwin County,AL,50,3,6,1,3,4117521611,1133190229,182265,...,141,1244,1198,46,0,0,0,104061,73180,30881
2,Barbour County,AL,50,3,6,1,5,2291818968,50864716,27457,...,5,135,132,2,0,0,1,11829,9820,2009
3,Bibb County,AL,50,3,6,1,7,1612480789,9289057,22915,...,8,120,119,1,0,0,0,8981,7953,1028
4,Blount County,AL,50,3,6,1,9,1669961855,15157440,57322,...,17,351,341,10,0,0,0,23887,21578,2309


In [36]:
print_sql(conn, "SELECT NAME, STUSAB, POP100 FROM us_counties_2010 LIMIT 10;")

> SELECT NAME, STUSAB, POP100 FROM us_counties_2010 LIMIT 10;
('Autauga County', 'AL', 54571)
('Baldwin County', 'AL', 182265)
('Barbour County', 'AL', 27457)
('Bibb County', 'AL', 22915)
('Blount County', 'AL', 57322)
('Bullock County', 'AL', 10914)
('Butler County', 'AL', 20947)
('Calhoun County', 'AL', 118572)
('Chambers County', 'AL', 34215)
('Cherokee County', 'AL', 25989)



### Math in SQL 

SQL can perform basic math operators. 

##### Addition, Subtraction, and Multiplication

In [37]:
print_sql(conn, 'SELECT 2 + 2;')

> SELECT 2 + 2;
(4,)



In [38]:
pretty_print_sql(conn, 'SELECT 3 * 4;')

> SELECT 3 * 4;


,3 * 4
0,12


##### Division and Modulo



In [39]:
pretty_print_sql(conn, 'SELECT 11 / 6;')

> SELECT 11 / 6;


,11 / 6
0,1


The result above may be unexpected.  Division on two integers returns only the integer quotient.  You can get the remainder with the modulo operator. 

In [40]:
pretty_print_sql(conn, 'SELECT 11 % 6;')

> SELECT 11 % 6;


,11 % 6
0,5


Here we can see division on floats.

In [41]:
pretty_print_sql(conn, 'SELECT 11.0 / 6;')

> SELECT 11.0 / 6;


,11.0 / 6
0,1.833333


#### Math on Table Columns

We will look at the census data already imported. 

The census tracks the population as well as the number of people who identified with one of six race categories or a combination of two or more races. 

The `AS` keyword, allows use to give each column a more readable alias in the result set. 


In [42]:
pretty_print_sql(conn, """
SELECT NAME,
       STUSAB AS "ST",
       P0010001 AS "Tot. Pop.",
       P0010003 AS "Caucasian Alone",
       P0010004 AS "African American Alone",
       P0010005 AS "Am Indian/Alaska Native Alone",
       P0010006 AS "Asian Alone",
       P0010007 AS "Native Hawaiian and Other Pacific Islander Alone",
       P0010008 AS "Some Other Race Alone",
       P0010009 AS "Two or More Races"
FROM us_counties_2010;
""")

> 
SELECT NAME,
       STUSAB AS "ST",
       P0010001 AS "Tot. Pop.",
       P0010003 AS "Caucasian Alone",
       P0010004 AS "African American Alone",
       P0010005 AS "Am Indian/Alaska Native Alone",
       P0010006 AS "Asian Alone",
       P0010007 AS "Native Hawaiian and Other Pacific Islander Alone",
       P0010008 AS "Some Other Race Alone",
       P0010009 AS "Two or More Races"
FROM us_counties_2010;



,NAME,ST,Tot. Pop.,Caucasian Alone,African American Alone,Am Indian/Alaska Native Alone,Asian Alone,Native Hawaiian and Other Pacific Islander Alone,Some Other Race Alone,Two or More Races
0,Autauga County,AL,54571,42855,9643,232,474,32,466,869
1,Baldwin County,AL,182265,156153,17105,1216,1348,89,3631,2723
2,Barbour County,AL,27457,13180,12875,114,107,29,894,258
3,Bibb County,AL,22915,17381,5047,64,22,13,185,203
4,Blount County,AL,57322,53068,761,307,117,38,2347,684
...,...,...,...,...,...,...,...,...,...,...
3138,Sweetwater County,WY,43806,38748,438,423,336,42,2799,1020
3139,Teton County,WY,21294,18821,49,111,235,15,1715,348
3140,Uinta County,WY,21118,19514,55,168,61,36,860,424
3141,Washakie County,WY,8533,7795,22,93,48,1,373,201


Now, we can do a simple calculation of adding to columns together. 

In [43]:
pretty_print_sql(conn, """
SELECT NAME,
       STUSAB AS "ST",
       P0010003 AS "Caucasian Alone",
       P0010004 AS "Afr. Amer. Alone",
       P0010003 + P0010004 AS "Total Causacian and Afr. Amer."
FROM us_counties_2010;
""")

> 
SELECT NAME,
       STUSAB AS "ST",
       P0010003 AS "Caucasian Alone",
       P0010004 AS "Afr. Amer. Alone",
       P0010003 + P0010004 AS "Total Causacian and Afr. Amer."
FROM us_counties_2010;



,NAME,ST,Caucasian Alone,Afr. Amer. Alone,Total Causacian and Afr. Amer.
0,Autauga County,AL,42855,9643,52498
1,Baldwin County,AL,156153,17105,173258
2,Barbour County,AL,13180,12875,26055
3,Bibb County,AL,17381,5047,22428
4,Blount County,AL,53068,761,53829
...,...,...,...,...,...
3138,Sweetwater County,WY,38748,438,39186
3139,Teton County,WY,18821,49,18870
3140,Uinta County,WY,19514,55,19569
3141,Washakie County,WY,7795,22,7817


We can use this to check the data.  The six race "alone" columns plus the "two or more races" column should add up to be the total. 

In [44]:
print_sql(conn, """
SELECT NAME,
       STUSAB AS "ST",
       P0010001 AS "Total",
       P0010003 + P0010004 + P0010005 + P0010006 + P0010007
              + P0010008 + P0010009 AS "All Races",
       (P0010003 + P0010004 + P0010005 + P0010006 + P0010007
              + P0010008 + P0010009) - P0010001 AS "Difference"
FROM us_counties_2010
ORDER BY "Difference" DESC;
""")

> 
SELECT NAME,
       STUSAB AS "ST",
       P0010001 AS "Total",
       P0010003 + P0010004 + P0010005 + P0010006 + P0010007
              + P0010008 + P0010009 AS "All Races",
       (P0010003 + P0010004 + P0010005 + P0010006 + P0010007
              + P0010008 + P0010009) - P0010001 AS "Difference"
FROM us_counties_2010
ORDER BY "Difference" DESC;

('Autauga County', 'AL', 54571, 54571, 0)
('Baldwin County', 'AL', 182265, 182265, 0)
('Barbour County', 'AL', 27457, 27457, 0)
('Bibb County', 'AL', 22915, 22915, 0)
('Blount County', 'AL', 57322, 57322, 0)
('Bullock County', 'AL', 10914, 10914, 0)
('Butler County', 'AL', 20947, 20947, 0)
('Calhoun County', 'AL', 118572, 118572, 0)
('Chambers County', 'AL', 34215, 34215, 0)
('Cherokee County', 'AL', 25989, 25989, 0)
('Chilton County', 'AL', 43643, 43643, 0)
('Choctaw County', 'AL', 13859, 13859, 0)
('Clarke County', 'AL', 25833, 25833, 0)
('Clay County', 'AL', 13932, 13932, 0)
('Cleburne County', 'AL', 14972, 14972, 0)
('Coffee County',

##### Finding Percentages 

Let's look at the percentage of the population that is Asian. 

We can calculate the percentage Asian, then sort by this value. 

In [45]:
print_sql(conn, """
SELECT DISTINCT NAME,
       STUSAB AS "ST",
       (CAST (P0010006 AS FLOAT) / P0010001) * 100 AS "pct_asian"
FROM us_counties_2010
ORDER BY "pct_asian" DESC;
""")

> 
SELECT DISTINCT NAME,
       STUSAB AS "ST",
       (CAST (P0010006 AS FLOAT) / P0010001) * 100 AS "pct_asian"
FROM us_counties_2010
ORDER BY "pct_asian" DESC;

('Honolulu County', 'HI', 43.894977691099626)
('Aleutians East Borough', 'AK', 35.97580388411334)
('San Francisco County', 'CA', 33.27165361664608)
('Santa Clara County', 'CA', 32.02237037519322)
('Kauai County', 'HI', 31.32461880132954)
('Aleutians West Census Area', 'AK', 28.879697896061856)
('Maui County', 'HI', 28.801813555162305)
('Alameda County', 'CA', 26.125112645346434)
('San Mateo County', 'CA', 24.791948233073654)
('Queens County', 'NY', 22.942661613594165)
('Hawaii County', 'HI', 22.179717850215315)
('Middlesex County', 'NJ', 21.39794877620521)
('Kodiak Island Borough', 'AK', 19.570335491465567)
('Orange County', 'CA', 17.865865488108557)
('Fairfax County', 'VA', 17.533183079633844)
('Fort Bend County', 'TX', 16.975443092035018)
('Fairfax city', 'VA', 15.209395080877467)
('Loudoun County', 'VA', 14.73947443413776

#### Aggregate Functions 

SQL also lets you calculate a result from values within a column with aggregate functions. 

For example, what is the total population of all counties or average population of all counties. 


In [46]:
pretty_print_sql(conn, """
SELECT sum(P0010001) AS "County Sum",
       round(avg(P0010001), 0) AS "County Average"
FROM us_counties_2010;
""")

> 
SELECT sum(P0010001) AS "County Sum",
       round(avg(P0010001), 0) AS "County Average"
FROM us_counties_2010;



,County Sum,County Average
0,308745538,98233.0


### Close Database Connections

When you are done executing your scripts you want to close the database connection. 

In [47]:
conn.close()
conn2.close()

In [ ]:
# Create two empty lists to store ham and spam data
ham_data = []
spam_data = []
# Specify the file path
file_path = "/home/campus19/trkosire/PycharmProjects/pythonProject/SMSSpamCollection"

import re
# Read the file line by line and separate ham and spam
try:
    with open(file_path, 'r') as file:
        for line in file:
            # Split each line into label and message
            parts = line.strip().split('\t')
            if len(parts) == 2:
                label, message = parts
                if label == "ham":
                    ham_data.append(message.split())
                    word_pattern = re.compile(r'\b\w+\b')
                    ham_data = [word for sublist in ham_data for text in sublist for word in word_pattern.findall(text)]              
                elif label == "spam":
                    spam_data.append(message.split())
                    spam_data = [word for sublist in spam_data for text in sublist for word in word_pattern.findall(text)]
except FileNotFoundError:
    print(f"The file '{file_path}' does not exist.")
except Exception as e:
    print(f"An error occurred: {e}")

# Now you have 'ham_data' and 'spam_data' lists containing the respective messages
print(f"Total ham messages: {len(ham_data)}")
print(f"Total spam messages: {len(spam_data)}")


In [30]:
ham_data

[['Go',
  'until',
  'jurong',
  'point,',
  'crazy..',
  'Available',
  'only',
  'in',
  'bugis',
  'n',
  'great',
  'world',
  'la',
  'e',
  'buffet...',
  'Cine',
  'there',
  'got',
  'amore',
  'wat...']]